## Radio interferometría y síntesis de imágenes en astronomía - Laboratorio 2

### Vicente Mieres

In [9]:
# Modulos
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [10]:
from modules.configurations import VLA_L_4
from modules.simulation import visibilities_simulation

V = visibilities_simulation(VLA_L_4)